*reference: https://janakiev.com/blog/pytorch-iris/
*reference: https://sofiadutta.github.io/datascience-ipynbs/pytorch/Image-Classification-using-PyTorch.html

In [157]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from torch.autograd import Variable

import matplotlib.pyplot as plt

from tqdm import tqdm

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [186]:
iris = load_iris()
X = iris['data']
Y = iris['target']
feature_names = iris['feature_names']

# Scale data to have mean 0 and variance 1 
# which is importance for convergence of the neural network
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data set into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

# # # Converting X and Y into tensors 
# X_train = Variable(torch.from_numpy(X_train)).float()
# y_train = Variable(torch.from_numpy(y_train)).long()
# X_test  = Variable(torch.from_numpy(X_test)).float()
# y_test  = Variable(torch.from_numpy(y_test)).long()
# X = Variable(torch.from_numpy(X)).long()


# Model Structure 
class model_structure(nn.Module):
    def __init__(self,input_dim):
        super(model_structure,self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim,30),
            nn.ReLU(),
            nn.Linear(30,3),
            nn.ReLU(),
            nn.Softmax(dim=1))
        
    def forward(self,x):
        out = self.net(x)
        return out 
    
# Hyperparameters
class Param(object):
    def __init__(self,epoch,batch_size,learning_rate,k_fold):
        self.epoch = epoch
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.k_fold = k_fold
        
args = Param(80,10,0.001,5)
model = model_structure(X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(),lr=args.learning_rate)
loss_fn = nn.CrossEntropyLoss()
model

# Initialize variables for tracking best model
best_model = None
best_accuracy = 0.0
loss_list = []
accuracy_list = []
epoch_loss_list = []
epoch_accuracy_list = []


cv = KFold(n_splits=args.k_fold, shuffle=True)

for train_index, val_index in cv.split(X):
    # test train split 
    # Split the data into training set and validation set for the current fold
    X_train, X_test = X[train_index], X[val_index]
    y_train, y_test = y[train_index], y[val_index]

    # Convert the data to PyTorch tensors
    X_train_tensor = torch.Tensor(X_train)
    y_train_tensor = torch.LongTensor(y_train)
    X_val_tensor = torch.Tensor(X_test)
    y_val_tensor = torch.LongTensor(y_test)
    
    # Create a PyTorch DataLoader for batch processing
    train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)

    for epoch in tqdm.trange(args.epoch):
        epoch_loss = 0.0
        epoch_correct = 0
        epoch_total = 0
        
        for inputs, labels in train_loader:
            # forward
            y_pred = model(inputs)
            loss = loss_fn(y_pred,labels)

            # Back-propagation and Resetting the optimizer 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Accumulate loss
            epoch_loss += loss.item() * inputs.size(0)

            # Calculate accuracy
            _, predicted = torch.max(y_pred,1)
            epoch_correct += (predicted == labels).sum().item()
            epoch_total += labels.size(0)

            epoch_loss /= len(train_loader.dataset)
            epoch_accuracy = epoch_correct / epoch_total

            epoch_loss_list.append(epoch_loss)
            epoch_accuracy_list.append(epoch_accuracy)

            loss_list.append(epoch_loss_list)
            accuracy_list.append(epoch_accuracy_list)


    with torch.no_grad():
        outputs = model(X_val_tensor)
        _, predicted = torch.max(outputs.data, 1)
        
        # Calculate accuracy for the current fold
        accuracy = accuracy_score(y_test, predicted)
        
        # Check if current model has higher accuracy than the best model
        if accuracy > best_accuracy:
            best_model = model
            best_accuracy = accuracy

# Convert the predictions and true labels to numpy arrays
predicted_np = predicted.numpy()
y_test_np = np.array(y_test)

# metrics 
"""
By setting average='macro', the metrics are calculated for each class independently, and then averaged. 
This approach treats all classes equally and can be useful when you want to evaluate the overall performance of the model across all classes.
"""
accuracy = accuracy_score(y_test_np, predicted_np)
precision = precision_score(y_test_np, predicted_np,average='macro',zero_division=0)
recall = recall_score(y_test_np, predicted_np,average='macro')
f1 = f1_score(y_test_np, predicted_np,average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

100%|██████████| 80/80 [00:00<00:00, 120.07it/s]

Accuracy: 0.9333333333333333
Precision: 0.9393939393939394
Recall: 0.9487179487179488
F1 Score: 0.9388888888888888


In [170]:
predicted_np

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [171]:
y_test_np

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])

In [176]:
torch.save(best_model.state_dict(), 'best-model-parameters.pt') # official recommended

In [123]:
model = model_structure(X_train.shape[1])
model.load_state_dict(torch.load('./best-model-parameters.pt'))

<All keys matched successfully>

In [185]:
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

best_model.eval()
X_test_tensor = torch.Tensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

with torch.no_grad():
    outputs = best_model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)

# Compute evaluation metrics
classification_rep = classification_report(y_test, predicted,labels=[0, 1, 2])
accuracy = accuracy_score(y_test, predicted)
print(classification_rep)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      0.92      0.96        12
           2       0.93      1.00      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.97      0.97        45
weighted avg       0.98      0.98      0.98        45

